## YOLOv1 구현

In [28]:
test = np.zeros((5))
idx = int(2)
test[idx] = 10
test

array([ 0.,  0., 10.,  0.,  0.])

In [14]:
import numpy as np
import cv2
from functools import partial
import xmltodict
from tqdm import tqdm
import tensorflow as tf
import random
from glob import glob

# 데이터 전처리

In [15]:
# 파일 경로
train_x_path = '/home/ubuntu/CUAI_2021/Advanced_Minkyu_Kim/PASCAL_VOC_2007/train/VOCdevkit/VOC2007/JPEGImages'
train_y_path = '/home/ubuntu/CUAI_2021/Advanced_Minkyu_Kim/PASCAL_VOC_2007/train/VOCdevkit/VOC2007/Annotations'

# 파일 경로 휙득
list_train_x = sorted([x for x in glob(train_x_path + '/**')])    
list_train_y = sorted([x for x in glob(train_y_path + '/**')]) 

image_file_path_list = sorted([x for x in glob(train_x_path + '/**')])
xml_file_path_list = sorted([x for x in glob(train_y_path + '/**')])

In [17]:
# 데이터셋에 존재하는 클래스가 얼마나 있는지 알아낸다
def get_Classes_inImage(xml_file_list):
    Classes_inDataSet = []

    for xml_file_path in xml_file_list: 

        f = open(xml_file_path)
        xml_file = xmltodict.parse(f.read())
        # 사진에 객체가 여러개 있을 경우
        try: 
            for obj in xml_file['annotation']['object']:
                Classes_inDataSet.append(obj['name'].lower()) # 들어있는 객체 종류를 알아낸다
        # 사진에 객체가 하나만 있을 경우
        except TypeError as e: 
            Classes_inDataSet.append(xml_file['annotation']['object']['name'].lower()) 
        f.close()

    Classes_inDataSet = list(set(Classes_inDataSet)) # set은 중복된걸 다 제거하고 유니크한? 아무튼 하나만 가져온다. 그걸 리스트로 만든다
    Classes_inDataSet.sort() # 정렬

    return Classes_inDataSet

In [18]:
# 이미지에 어떤 Ground Truth Box가 있는지(label 휙득)
def get_label_fromImage(xml_file_path, Classes_inDataSet):

    f = open(xml_file_path)
    xml_file = xmltodict.parse(f.read()) 

    Image_Height = float(xml_file['annotation']['size']['height'])
    Image_Width  = float(xml_file['annotation']['size']['width'])

    label = np.zeros((7, 7, 25), dtype = np.float32)
    
    try:
        for obj in xml_file['annotation']['object']:
            
            # class의 index 휙득
            class_index = Classes_inDataSet.index(obj['name'].lower())
            
            # min, max좌표 얻기
            x_min = float(obj['bndbox']['xmin']) 
            y_min = float(obj['bndbox']['ymin'])
            x_max = float(obj['bndbox']['xmax']) 
            y_max = float(obj['bndbox']['ymax'])

            # 224*224에 맞게 변형시켜줌
            x_min = float((224.0/Image_Width)*x_min)
            y_min = float((224.0/Image_Height)*y_min)
            x_max = float((224.0/Image_Width)*x_max)
            y_max = float((224.0/Image_Height)*y_max)

            # 변형시킨걸 x,y,w,h로 만들기 
            x = (x_min + x_max)/2.0
            y = (y_min + y_max)/2.0
            w = x_max - x_min
            h = y_max - y_min

            # x,y가 속한 cell알아내기
            x_cell = int(x/32) # 0~6
            y_cell = int(y/32) # 0~6
            x_val_inCell = float((x - x_cell * 32.0)/32.0) # 0.0 ~ 1.0
            y_val_inCell = float((y - y_cell * 32.0)/32.0) # 0.0 ~ 1.0

            # w, h 를 0~1 사이의 값으로 만들기
            w = w / 224.0
            h = h / 224.0

            class_index_inCell = class_index + 5

            label[y_cell][x_cell][0] = x_val_inCell
            label[y_cell][x_cell][1] = y_val_inCell
            label[y_cell][x_cell][2] = w
            label[y_cell][x_cell][3] = h
            label[y_cell][x_cell][4] = 1.0
            label[y_cell][x_cell][class_index_inCell] = 1.0


    # single-object in image
    except TypeError as e : 
        # class의 index 휙득
        class_index = Classes_inDataSet.index(xml_file['annotation']['object']['name'].lower())
            
        # min, max좌표 얻기
        x_min = float(xml_file['annotation']['object']['bndbox']['xmin']) 
        y_min = float(xml_file['annotation']['object']['bndbox']['ymin'])
        x_max = float(xml_file['annotation']['object']['bndbox']['xmax']) 
        y_max = float(xml_file['annotation']['object']['bndbox']['ymax'])

        # 224*224에 맞게 변형시켜줌
        x_min = float((224.0/Image_Width)*x_min)
        y_min = float((224.0/Image_Height)*y_min)
        x_max = float((224.0/Image_Width)*x_max)
        y_max = float((224.0/Image_Height)*y_max)

        # 변형시킨걸 x,y,w,h로 만들기 
        x = (x_min + x_max)/2.0
        y = (y_min + y_max)/2.0
        w = x_max - x_min
        h = y_max - y_min

        # x,y가 속한 cell알아내기
        x_cell = int(x/32) # 0~6
        y_cell = int(y/32) # 0~6
        x_val_inCell = float((x - x_cell * 32.0)/32.0) # 0.0 ~ 1.0
        y_val_inCell = float((y - y_cell * 32.0)/32.0) # 0.0 ~ 1.0

        # w, h 를 0~1 사이의 값으로 만들기
        w = w / 224.0
        h = h / 224.0

        class_index_inCell = class_index + 5

        label[y_cell][x_cell][0] = x_val_inCell
        label[y_cell][x_cell][1] = y_val_inCell
        label[y_cell][x_cell][2] = w
        label[y_cell][x_cell][3] = h
        label[y_cell][x_cell][4] = 1.0
        label[y_cell][x_cell][class_index_inCell] = 1.0

    return label # np array로 반환


In [29]:
# 데이터 증강을 할거면 여기서 해야한다.
def make_dataset(image_file_path_list, xml_file_path_list) :

    Classes_inDataSet = get_Classes_inImage(xml_file_path_list)

    image_dataset = 0
    label_dataset = 0

    for i in tqdm(range(0, len(image_file_path_list)), desc = "make dataset"):
        image = cv2.imread(image_file_path_list[i]) / 255.0 # 이미지를 넘파이 배열로 불러온 뒤 255로 나눠 픽셀별 R, G, B를 0~1사이의 값으로 만들어버린다.
        label = get_label_fromImage(xml_file_path_list[i], Classes_inDataSet)
        
        # 여기서 데이터 증강을 시도해야한다고 생각한다
        # 랜덤한 값을 뽑아내고 만약 그 값이 0.5를 넘기면 데이터 증강의 대상이 되는 이미지가 되는거다.


        if image_dataset == 0 : 
            image_dataset = image.copy()
            label_dataset = label.copy()
        else : 
            image_dataset = np.append(image_dataset, image)
            label_dataset = np.append(label_dataset, label)
    

    image_dataset = np.reshape(image_dataset, (-1, 224, 224, 3))
    label_dataset = np.reshape(label_dataset, (-1, 7, 7, 25))

    return image_dataset, label_dataset


## Data Augmentation
출처 : https://towardsdatascience.com/complete-image-augmentation-in-opencv-31a6b02694f5

In [19]:
def fill(img, h, w):
    img = cv2.resize(img, (h, w), cv2.INTER_CUBIC)
    return img

# 0.2로 설정하면 될듯(up to 20% of the original image size라고 해서)
def horizontal_shift(img, ratio=0.0):
    if ratio > 1 or ratio < 0:
        print('Value should be less than 1 and greater than 0')
        return img
    ratio = random.uniform(-ratio, ratio)
    h, w = img.shape[:2]
    to_shift = w*ratio
    if ratio > 0:
        img = img[:, :int(w-to_shift), :]
    if ratio < 0:
        img = img[:, int(-1*to_shift):, :]
    img = fill(img, h, w)
    return img

def vertical_shift(img, ratio=0.0):
    if ratio > 1 or ratio < 0:
        print('Value should be less than 1 and greater than 0')
        return img
    ratio = random.uniform(-ratio, ratio)
    h, w = img.shape[:2]
    to_shift = h*ratio
    if ratio > 0:
        img = img[:int(h-to_shift), :, :]
    if ratio < 0:
        img = img[int(-1*to_shift):, :, :]
    img = fill(img, h, w)
    return img

def zoom(img, value): # 전체 이미지의 value(0~1)만큼만 가져가는거니까 20%면 0.8로?
    if value > 1 or value < 0:
        print('Value for zoom should be less than 1 and greater than 0')
        return img
    value = random.uniform(value, 1)
    h, w = img.shape[:2]
    h_taken = int(value*h)
    w_taken = int(value*w)
    h_start = random.randint(0, h-h_taken)
    w_start = random.randint(0, w-w_taken)
    img = img[h_start:h_start+h_taken, w_start:w_start+w_taken, :]
    img = fill(img, h, w)
    return img


In [20]:
# 밝기 조정(The more the value of Saturation and Value matrices the greater is the brightness)

def brightness(img, low, high): # low = 0.5, high = 1.5로 설정하면 될듯?
    value = random.uniform(low, high) # low~high 사이 랜덤한 값을 기존 saturation에다 곱한다
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype = np.float64)
    hsv[:,:,1] = hsv[:,:,1]*value
    hsv[:,:,1][hsv[:,:,1]>255]  = 255
    hsv[:,:,2] = hsv[:,:,2]*value 
    hsv[:,:,2][hsv[:,:,2]>255]  = 255
    hsv = np.array(hsv, dtype = np.uint8)
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return img

In [21]:
max_num = len(tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3)).layers) # 레이어 최대 개수

YOLO = tf.keras.models.Sequential(name = "YOLO")
for i in range(0, max_num-1):
    YOLO.add(tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3)).layers[i])

initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)
regularizer = tf.keras.regularizers.l2(0.0005) # L2 규제 == weight decay

for layer in YOLO.layers:
    # 'kernel_regularizer' 속성이 있는 인스턴스를 찾아 regularizer를 추가
    if hasattr(layer, 'kernel_regularizer'):
        setattr(layer, 'kernel_regularizer', regularizer)

### 원문은 DarkNet을 썼지만 나는 구현을 쉽게 하기 위해 VGG16을 썼다.
### 여기에 따로 레이어를 얹어서 YOLO를 구현할거다

In [22]:
YOLO.add(tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer=initializer, padding = 'SAME' ,kernel_regularizer = regularizer, name = "detection_conv1", dtype='float32'))
YOLO.add(tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer=initializer, padding = 'SAME' ,kernel_regularizer = regularizer, name = "detection_conv2", dtype='float32'))
YOLO.add(tf.keras.layers.MaxPool2D((2, 2)))
YOLO.add(tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer=initializer, padding = 'SAME' ,kernel_regularizer = regularizer, name = "detection_conv3", dtype='float32'))
YOLO.add(tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer=initializer, padding = 'SAME' ,kernel_regularizer = regularizer, name = "detection_conv4", dtype='float32'))
# Linear 부분
YOLO.add(tf.keras.layers.Flatten())
YOLO.add(tf.keras.layers.Dense(4096, activation= None, kernel_initializer = initializer, name = "detection_linear1", dtype='float32'))
YOLO.add(tf.keras.layers.Dropout(.5))
YOLO.add(tf.keras.layers.Dense(1470, activation=partial(tf.nn.leaky_relu, alpha=0.01), kernel_initializer = initializer, name = "detection_linear2", dtype='float32')) # 7*7*30 = 1470. 0~29 : (0, 0) 위치의 픽셀에 대한 각종 출력값, 30~59 : (1, 0) 위치의...블라블라
YOLO.add(tf.keras.layers.Reshape((7, 7, 30), name = 'output', dtype='float32'))

In [23]:
YOLO.summary()

Model: "YOLO"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168 

In [24]:
def yolo_multitask_loss(y_true, y_pred): # 커스텀 손실함수
    
        # 계산을 위해 y_true을 tensor로 만들어준다. 
        y_true = tf.convert_to_tensor(y_true)

        # box = [x,y,w,h,confidence_score]
        pred_box_batch_1 = y_pred[..., :4] # [-1,7,7,4]
        pred_confidence_batch_1 = y_pred[..., 4] # [-1,7,7,1]
        pred_box_batch_2 = y_pred[..., 5:9] # [-1,7,7,4]
        pred_confidence_batch_2 = y_pred[..., 9] # [-1,7,7,1]
        pred_class_batch = y_pred[..., 10:] # [-1,7,7,20]

        # y_true는 ground truth box(0~4) + class(5~24)로 구성되어 있다 
        true_box_batch = y_true[..., :4] # [-1,7,7,5]
        true_confidence_batch = y_true[..., 4] # [-1,7,7,1]
        true_class_batch = y_true[..., 5:] # [-1,7,7,20]

        # YOLOv1의 Loss function은 3개로 나뉜다. localization, confidence, classification
        # localization은 추측한 box랑 ground truth box의 오차
        # confidencee는 Pr(Object) * IOU, classification은 Pr(Class|Object) = 객체가 있을 때 해당 객체일 확률이다. 어떻게 계산하지?

        loss = tf.Variable(0.0)
        count = 0.0
        # loss를 계산 
        for boxes_pred_1, confidences_pred_1, boxes_pred_2, confidences_pred_2, class_pred, boxes_true, confidences_true, class_true in pred_box_batch_1, pred_confidence_batch_1, pred_box_batch_2, pred_confidence_batch_2, pred_class_batch, true_box_batch, true_confidence_batch, true_class_batch :
                # boxes_pred_1 : [7,7,4], boxes_pred_2 : [7,7,4], boxes_true : [7,7,4], class_pred : [7,7,20], class_true : [7,7,20], pred_confidences_1 : [7, 7, 1], pred_confidences_2 : [7, 7, 1]
                boxes_pred_1 = tf.reshape(boxes_pred_1, [49, 4])
                confidences_pred_1 = tf.reshape(confidences_pred_1, [49, 1])
                boxes_pred_2 = tf.reshape(boxes_pred_2, [49, 4])
                confidences_pred_2 = tf.reshape(confidences_pred_2, [49, 1])
                class_pred = tf.reshape(boxes_pred_2, [49, 20])

                boxes_true = tf.reshape(boxes_true, [49, 4])
                confidences_true = tf.reshape(confidences_true, [49, 1])
                class_true = tf.reshape(class_true, [49, 20])

                for box_pred_1, confidence_pred_1, box_pred_2, confidence_pred_2, class_pred_oneCell, box_true, confidence_true, class_true_oneCell in boxes_pred_1, confidences_pred_1, boxes_pred_2, confidences_pred_2, class_pred, boxes_true, confidences_true, class_true :
                        # 한 셀에서 responsible한 box를 골라 localization error를 구해야함. reponsible한 box가 아니면 손실값은 0이 되며 confidence loss, classification loss를 구할 때도 reponsible한 box일 경우에만 계산을 수행함.
                        # responsible한 box는 IoU를 기준으로 판단함

                        # IoU 구하기
                        # x,y,w,h -> min_x, min_y, max_x, max_y로 변환
                        box_pred_1_np = box_pred_1.numpy()
                        box_pred_2_np = box_pred_2.numpy()
                        box_true_np = box_true.numpy()

                        box_pred_1_area = box_pred_1_np[2] * box_pred_1_np[3]
                        box_pred_2_area = box_pred_2_np[2] * box_pred_2_np[3]
                        box_true_area  = box_true_np[2]  * box_true_np[3]

                        box_pred_1_minmax = np.asarray([box_pred_1_np[0] - 0.5*box_pred_1_np[2], box_pred_1_np[1] - 0.5*box_pred_1_np[3], box_pred_1_np[0] + 0.5*box_pred_1_np[2], box_pred_1_np[1] + 0.5*box_pred_1_np[3]])
                        box_pred_2_minmax = np.asarray([box_pred_2_np[0] - 0.5*box_pred_2_np[2], box_pred_2_np[1] - 0.5*box_pred_2_np[3], box_pred_2_np[0] + 0.5*box_pred_2_np[2], box_pred_2_np[1] + 0.5*box_pred_2_np[3]])
                        box_true_minmax = np.asarray([box_true_np[0] - 0.5*box_true_np[2], box_true_np[1] - 0.5*box_true_np[3], box_true_np[0] + 0.5*box_true_np[2], box_true_np[1] + 0.5*box_true_np[3]])

                        # 곂치는 영역의 (min_x, min_y, max_x, max_y)
                        InterSection_pred_1_with_true = [max(box_pred_1_minmax[0], box_true_minmax[0]), max(box_pred_1_minmax[1], box_true_minmax[1]), min(box_pred_1_minmax[2], box_true_minmax[2]), min(box_pred_1_minmax[3], box_true_minmax[3])]
                        InterSection_pred_2_with_true = [max(box_pred_2_minmax[0], box_true_minmax[0]), max(box_pred_2_minmax[1], box_true_minmax[1]), min(box_pred_2_minmax[2], box_true_minmax[2]), min(box_pred_2_minmax[3], box_true_minmax[3])]

                        # 박스별로 IoU를 구한다
                        IntersectionArea_pred_1_true = 0

                        # 음수 * 음수 = 양수일 수도 있으니 검사를 한다.
                        if (InterSection_pred_1_with_true[2] - InterSection_pred_1_with_true[0] + 1) >= 0 and (InterSection_pred_1_with_true[3] - InterSection_pred_1_with_true[1] + 1) >= 0 :
                                IntersectionArea_pred_1_true = (InterSection_pred_1_with_true[2] - InterSection_pred_1_with_true[0] + 1) * InterSection_pred_1_with_true[3] - InterSection_pred_1_with_true[1] + 1

                        IntersectionArea_pred_2_true = 0

                        if (InterSection_pred_2_with_true[2] - InterSection_pred_2_with_true[0] + 1) >= 0 and (InterSection_pred_2_with_true[3] - InterSection_pred_2_with_true[1] + 1) >= 0 :
                                IntersectionArea_pred_2_true = (InterSection_pred_2_with_true[2] - InterSection_pred_2_with_true[0] + 1) * InterSection_pred_2_with_true[3] - InterSection_pred_2_with_true[1] + 1

                        Union_pred_1_true = box_pred_1_area + box_true_area - IntersectionArea_pred_1_true
                        Union_pred_2_true = box_pred_2_area + box_true_area - IntersectionArea_pred_2_true

                        IoU_box_1 = IntersectionArea_pred_1_true/Union_pred_1_true
                        IoU_box_2 = IntersectionArea_pred_2_true/Union_pred_2_true
                        
                        responsible_IoU = 0
                        responsible_box = 0
                        responsible_bbox_confidence = 0
                        nonresponsible_bbox_confidence = 0

                        # box1, box2 중 responsible한걸 선택(IoU 기준)
                        if IoU_box_1 >= IoU_box_2 :
                                responsible_IoU = IoU_box_1
                                responsible_box = tf.identity(box_pred_1)
                                responsible_bbox_confidence = tf.identity(confidence_pred_1)
                                non_responsible_bbox_confidence = tf.identity(confidence_pred_2)
                                
                        else :
                                responsible_IoU = IoU_box_2
                                responsible_box = tf.identity(box_pred_2)
                                responsible_bbox_confidence = tf.identity(confidence_pred_2)
                                non_responsible_bbox_confidence = tf.identity(confidence_pred_1)
                        
                        # 만약 해당 cell에 객체가 없으면 confidence error의 no object 파트만 판단. (label된 값에서 알아서 해결)
                        # 0~3 : bbox1의 위치 정보, 4 : bbox1의 bbox confidence score, 5~8 : bbox2의 위치 정보, 9 : bbox2의 confidence score, 10~29 : cell에 존재하는 클래스 확률 = pr(class | object) 

                        # localization error 구하기(x,y,w,h). x, y는 해당 grid cell의 중심 좌표와 offset이고 w, h는 전체 이미지에 대해 정규화된 값이다. 즉, 범위가 0~1이다.
                        localization_err_x = tf.math.pow( tf.math.subtract(box_true[0], responsible_box[0]), 2) # (x-x_hat)^2
                        localization_err_y = tf.math.pow( tf.math.subtract(box_true[1], responsible_box[1]), 2) # (y-y_hat)^2

                        localization_err_w = tf.math.pow( tf.math.subtract(tf.math.pow(box_true[2], 0.5), tf.math.pow(responsible_box[2], 0.5)), 2) # (sqrt(w) - sqrt(w_hat))^2
                        localization_err_h = tf.math.pow( tf.math.subtract(tf.math.pow(box_true[3], 0.5), tf.math.pow(responsible_box[3], 0.5)), 2) # (sqrt(h) - sqrt(h_hat))^2
                        
                        localization_err_1 = tf.math.add(localization_err_x, localization_err_y)
                        localization_err_2 = tf.math.add(localization_err_w, localization_err_h)
                        localization_err = tf.math.add(localization_err_1, localization_err_2)
                        weighted_localization_err = tf.math.multiply(localization_err, 5.0) # 5.0 : λ_coord
                        # confidence error 구하기. true의 경우 답인 객체는 1 * ()고 아니면 0*()가 된다. 
                        # index 4, 9에 있는 값(0~1)이 해당 박스에 객체가 있을 확률을 나타낸거다. Pr(obj in bbox)
                        class_confidence_score = tf.math.add(tf.math.pow(tf.math.subtract(responsible_bbox_confidence, confidence_true), 2), 
                                                             tf.math.multiply(tf.math.pow(tf.math.subtract(non_responsible_bbox_confidence, confidence_true), 2), 0.5) ) 
                        

                        # classification loss(10~29. 인덱스 10~29에 해당되는 값은 Pr(Classi |Object)이다. 객체가 cell안에 있을 때 해당 객체일 확률
                        # class_true_oneCell는 진짜 객체는 1이고 나머지는 0일거다. 
                        classification_err = tf.reduce_sum(tf.math.multiply(tf.math.subtract(class_true_oneCell, class_pred_oneCell), tf.math.subtract(class_true_oneCell, class_pred_oneCell)))

                        # loss합체
                        loss_OneCell_1 = tf.math.add(weighted_localization_err, class_confidence_score)
                        loss_OneCell = tf.math.add(loss_OneCell_1, classification_err)
                        loss = tf.math.add(loss, loss_OneCell)
                count = count + 1.0
        
        # 배치에 대한 loss 구하기
        tot_loss = tf.math.divide(loss, count)

        return tot_loss

# 훈련 계획
## epoch : 135
## batch size : 64
## momentum : 0.9
## weight decay : 0.0005
## learning rate : 0.01(1~75), 0.001(76~105), 0.0001(106~135)

## data augmentation : random scaling, translation을 원래 이미지 사이즈의 20%까지만 수행

In [25]:
# "We continue training with 10−2 for 75 epochs, then 10−3 for 30 epochs, and finally 10−4 for 30 epochs" 구현

LR_SCHEDULE = [
    # (epoch to start, learning rate) tuples
    (0, 1e-2), (75, 1e-3), (105, 1e-4)
]

def lr_schedule(epoch, lr):
  """Helper function to retrieve the scheduled learning rate based on epoch."""
  if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
    return lr
  for i in range(len(LR_SCHEDULE)):
    if epoch == LR_SCHEDULE[i][0]:
      return LR_SCHEDULE[i][1]
  return lr


In [26]:
optimizer = tf.keras.optimizers.RMSprop(lr = 1e-2, momentum=0.9)

YOLO.compile(loss = yolo_multitask_loss, optimizer=optimizer)

YOLO.fit(x_train, y_train,
          batch_size=64,
          epochs=135,
          verbose=0,
          callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_schedule)])

NameError: name 'x_train' is not defined